# Trabalho 1 de Cálculo Numérico
### Vítor Amorim Fróis
### 12543440

# 1) Sistemas Lineares com Matrizes Simétricas


In [1]:
import numpy as np

#### Considere a Matriz $A$ e o vetor $b$ dados. Considere $n$ a dimensão do problema.

#### a) Escreva um código que monte a matriz A para n = 1000.


In [2]:
n = 1000
tol = 10e-20

In [3]:
def create_band_matrix(n):
  band_matrix = np.zeros((n,n))
  for i in range(n):
    for j in range(n):
      if(abs(i-j) == 0):
        band_matrix[i, j] = 100
      elif(abs(i-j) == 1):
        band_matrix[i, j] = -4
      elif(abs(i-j) == 2):
        band_matrix[i, j] = 1
  return band_matrix

In [4]:
A = create_band_matrix(n)
A

array([[100.,  -4.,   1., ...,   0.,   0.,   0.],
       [ -4., 100.,  -4., ...,   0.,   0.,   0.],
       [  1.,  -4., 100., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ..., 100.,  -4.,   1.],
       [  0.,   0.,   0., ...,  -4., 100.,  -4.],
       [  0.,   0.,   0., ...,   1.,  -4., 100.]])

In [5]:
b = np.ones(n) / (n**4)

#### b) Escreva um código que faça a decomposição de Cholesky de uma matriz simétrica definida positiva qualquer

In [6]:
def cholesky(A):
  n = np.shape(A)[0]
  L = np.zeros((n,n))
  for i in np.arange(n):
    for j in np.arange(i+1):
      if i == j:
        L[i, i] = np.sqrt(A[i, i] - np.sum(L[i, :i] ** 2))
      else:
        L[i, j] = (A[i, j] - np.sum(L[i, :j] * L[j, :j])) / L[j, j]

  return L

In [7]:
A_cholesky = cholesky(A)
A_cholesky

array([[10.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.4       ,  9.9919968 ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.1       , -0.39631718,  9.99164314, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  9.99164043,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ..., -0.39636438,
         9.99164043,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.10008367,
        -0.39636438,  9.99164043]])

#### c) Usando as rotinas implementadas anteriormente, escreva um código para resolver um sistema $Ax = b$

In [8]:
def gauss_siedel(A, b, tol):
  n = np.shape(A)[0]
  x0 = np.zeros(n)
  L = np.tril(A)
  R = np.triu(A, 1)
  C = np.linalg.solve(-L, R)
  g = np.linalg.solve(L, b)
  kmax = 10000; k = 0;
  while(np.linalg.norm(b-A@x0) > tol and k < kmax):
    k = k+1
    x0 = (C @ x0) + g
  if k == kmax:
    raise Exception('Method does not converge')
  return x0

In [9]:
def solve_system(A, b):
  L = cholesky(A)
  y = np.linalg.solve(L, b)
  x = np.linalg.solve(L.T, y)
  return x

In [10]:
x = solve_system(A, b)

In [11]:
method_error = np.linalg.norm((A @ b)-x)
method_error

2.9723344551319323e-09

#### d) Escreva um código implementando o método de Jacobi para resolver um sistema $Ax = b$. Utilize o erro absoluto como critério de parada.


In [12]:
def gauss_jacobi(A,b,tol):
  n = np.shape(A)[0]
  D = np.diag(np.diag(A))
  aux_C = np.linalg.solve(D,A)
  C = np.eye(n) - aux_C
  g = np.linalg.solve(D,b)
  kmax = 10000 
  k = 0
  x0 = np.ones(n)
  old_x0 = np.zeros(n)
  
  error = 10

  while error>tol and k<kmax:
      error = np.linalg.norm(x0 - old_x0)
      k = k+1
      old_x0 = x0
      x0 = C.dot(x0)+g

  x = x0
  return x,k,error

In [13]:
x, iterations, jacobi_error = gauss_jacobi(A, b, method_error)
jacobi_error

3.016904157156495e-10

#### e) Vamos comparar o método direto de Cholesky com o iterativo de Jacobi neste exemplo. Observe quanto tempo leva para resolver o sistema usando Cholesky. Quantas iterações foram necessárias no método de Jacobi para obtermos a mesma precisão da solução dada pelo método de Cholesky ?


In [14]:
import time

In [15]:
# Cholesky method
time_mean = 0
iterations = 10
for i in range(iterations):
  start = time.time()
  x = solve_system(A, b)
  end = time.time()
  time_mean += (end-start)
time_mean /= iterations
cholesky_error = np.linalg.norm((A@b) - x) 
print(f"Resultado após {iterations} iterações: ")
print(f"Tempo médio: {time_mean:.2f}s")
print(f"Erro: {cholesky_error}")

Resultado após 10 iterações: 
Tempo médio: 3.70s
Erro: 2.9723344551319323e-09


In [16]:
# Jacobi method 
time_mean = 0
iterations = 10
for i in range(iterations):
  start = time.time()
  x, k, jacobi_error = gauss_jacobi(A, b, cholesky_error)
  end = time.time()
  time_mean += (end-start)
time_mean /= iterations
print(f"Resultado após as iterações: ")
print(f"Tempo médio: {time_mean:.2f}s")
print(f"Iterações necessárias: {k}")
print(f"Erro: {jacobi_error}")

Resultado após as iterações: 
Tempo médio: 0.10s
Iterações necessárias: 11
Erro: 3.016904157156495e-10


#### f)  É possível melhorar a implementação da decomposição de Cholesky para o exemplo em questão?

Sim, é possível. Dado que estamos fazendo operações em um sistema com matriz de banda podemos ignorar os zeros para tornar o processamento mais rápido.

# 2) Método de Newton para Sistemas Não-Lineares

$$ x^{k+1} = x^k - J_F(x^k)^{-1}F(x^k) $$

### 1) Observando a equação acima, vê-se a necessidade de calcular a matriz inversa da Jacobiana.
E possível calcular matrizes inversas usando a decomposição LU, sendo assim, implemente um código que:





#### a) Calcule a fatoração LU de uma matriz qualquer;

In [17]:
def decompLU(A):
  n = np.shape(A)[0]
  L = np.eye(n) 
  U = np.zeros((n,n))
  for k  in np.arange(n):
      for j in np.arange(k,n):
        U[k,j]=A[k,j]
        for s in np.arange(k):
            U[k,j] = U[k,j] - L[k,s]*U[s,j]
      for i in np.arange(k+1,n):
        L[i,k]=A[i,k]
        for s in np.arange(k):
            L[i,k] = L[i,k] - L[i,s]*U[s,k]
        L[i,k] = L[i,k]/U[k,k]
  return L, U  

In [18]:
A = np.random.randint(1, 100, (3, 3))
L, U = decompLU(A)

#### b) Resolva um sistema linear cujo lado direito é uma matriz;


Nesse exercício vamos resolver as partes superior e inferior de um sistema. Para isso, usamos as funções abaixo:

In [19]:
def solve_triangular_inferior(A, b):
  n = len(b)
  if len(A[0]) != n:
    raise ValueError(f'Matrix sides length does not match! {len(A[0])} x {len(b)}')

  x = np.zeros((n, 1))

  for i in range(n):
    s = 0
    for j in range(i):
      s += A[i][j]*x[j]

    x[i] = (b[i] - s)/(A[i][i])

  return x

In [20]:
def solve_triangular_superior(A, b):
  n = len(b)
  if len(A[0]) != n:
    raise ValueError(f'Matrix sides length does not match! {len(A[0])} x {len(b)}')

  x = np.zeros((n, 1))

  for i in range(n-1, 0, -1):
    s = 0
    for j in range(i+1, n+1, 1):
      s += A[i-1][j-1]*x[j-1]

    x[i-1] = (b[i-1] - s)/(A[i-1][i-1])

  return x

Montamos um sistema aleatório e a função para resolver, em duas partes.

In [21]:
A = np.random.randint(1, 100, (3, 3))
b = np.random.randint(1, 100, (3, 3))

In [22]:
def solve_matrix_system(A, b):
    y = np.linalg.solve(L, b)
    x = np.linalg.solve(U, y)
    return x

In [14]:
x = solve_matrix_system(A, b)
print(f'Erro da função: {np.linalg.norm(A @ x - b)}')
print(f(x))

NameError: name 'A' is not defined

#### c) Finalmente, calcule a inversa de uma matriz qualquer resolvendo vários sistemas lineares.

Sabemos que $A\times A^{-1}=I$, então montamos um sistema do tipo $A\times X=I$ para obter $X$.

Nos aproveitamos da propriedade 

In [ ]:
def inverse(A):
    return solve_matrix_system(A, np.eye(A.shape[0]))

In [ ]:
inv_A = solve_matrix_system(A, np.eye(A.shape[0]))
print(abs(np.round(A @ inv_A)))

In [ ]:
print(f'Erro da função inversa: {np.linalg.norm(np.eye(A.shape[0]) - (A @ inv_A))}')

### 2) Implemente o método de Newton para sistemas usando a rotina implementada no item anterior.
Lembre-se de especificar o critério de parada utilizado.

In [ ]:
def solve_system(A, b):
    L, U = decompLU(A)
    y = solve_triangular_inferior(L, b[:,k])
    x = solve_triangular_superior(U, y)
    return x

Para testar, definimos o sistema 
$$x²+y²=2$$
$$x²-\frac{y²}{9}=1$$

In [ ]:
def f(X):
  x1, x2 = X[0], X[1]
  f1 = x1**2 + x2**2 - 2
  f2 = x1**2 - (x2**2)/9 - 1
  return np.array([f1, f2])

In [ ]:
def jac(X):
  x1, x2 = X[0], X[1]
  diff_1 = np.array([2*x1, 2*x2])
  diff_2 = np.array([2*x1, -2*x2/9])
  return np.array([diff_1, diff_2])

Também vamos usar uma pequena ferramenta para checar se a solução do problema está correta

In [ ]:
def assert_method(method, x0, f, jac):
    x = method(x0, f, jac)
    zeros = f(x)
    tol = 10e-10
    norm = np.linalg.norm(zeros - np.zeros_like(zeros))
    if norm < tol:
        print(f'A solução do sistema f é {x}.')
    else:
        print('A solução não existe ou não pode ser calculada.')

O sistema para resolver equações não lineares proposto é da forma

$$x^{k+1} = x^k − J_F(x^k)^{−1}F(x^k)$$

In [ ]:
def newton_system2(x0, F, jac, tol=10e-5, kmax=1000):
  x = x0
  for k in range(kmax):
    j_x_inv = np.linalg.inv(jac(x))
    F_x = F(x)
    v = j_x_inv @ F_x
    x = x - v
    if np.linalg.norm(v) < tol:
      break

  return x

In [ ]:
x0 = np.array([2, 2])
x = newton_system2(x0, f, jac)

Verificamos se a solução está certa.

In [ ]:
assert_method(newton_system2, x0, f, jac)

### 3) Implemente novamente o método de Newton para sistemas usando o algoritmo anterior. 
Utilize as rotinas já implementadas neste trabalho para resolução do sistema linear, indicando sempre
qual está usando.

In [ ]:
def newton_system3(x0, F, jac, tol=10e-10, kmax=10000):
  x = x0
  for k in range(kmax):
    j_x = (jac(x))
    F_x = F(x)
    z = solve_matrix_system(j_x, F_x)
    x = x - z
    if np.linalg.norm(z) < tol:
      break

  return x

In [ ]:
x = newton_system3(x0, f, jac)

In [ ]:
assert_method(newton_system3, x0, f, jac)

### 4) Dada a equação de um círculo (x − a)2 + (y − b)2 = R2 e três pontos que passam por esse círculo

#### a) Monte um sistema não-linear para determinar a, b e R

\begin{align}
(8.21 - a)²+b²=R² \tag{1} \\
(0.34-a)²-(6.62-b)²=R² \tag{2} \\
(5.96-a)²-(-1.12-b)²=R² \tag{3}
\end{align}

In [ ]:
def func(X):
  a, b, R = X[0], X[1], X[2]
  f1 = (8.21 - a)**2 + b**2 - R**2
  f2 = (0.34 - a)**2 + (6.62 - b)**2 - R**2
  f3 = (5.96 - a)**2 + (-1.12 - b)**2 - R**2
  return np.array([f1, f2, f3])

In [ ]:
def jac(X):
  a, b, R = X[0], X[1], X[2]
  f1 = np.array([2*(a - 8.21), 2*(b), -2*R])
  f2 = np.array([2*(a - 0.34), 2*(b - 6.62), -2*R])
  f3 = np.array([2*(a - 5.96), 2*(b + 1.12), -2*R])
  return np.array([f1, f2, f3])

In [ ]:
x0 = np.array([1.0, 2.0, 3.0])

#### b) Resolva o sistema não-linear utilizando os códigos feitos nos itens 2 e 3

In [ ]:
start = time.time()
print(assert_method(newton_system2, x0, func, jac))
end = time.time()
print(f'Tempo gasto: {end-start}s')

In [ ]:
start = time.time()
print(assert_method(newton_system3, x0, func, jac))
end = time.time()
print(f'Tempo gasto: {end-start}s')

#### c) Houve melhora no tempo de execução do código implementado no item 3?

Observando acima o tempo gasto por ambas implementações do método de Newton, observa-se que o método sem inversa obteve desempenho 3 vezes melhor.

#### d) Finalmente, com os resultados a, b e R, utilizando a equação do círculo dada por $(x − a)^2 + (y − b)^2 = R^2$, imprima o gráfico que o representa.

In [ ]:
circle_x, circle_y, circle_r = newton_system3(x0, func, jac)

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(circle_x - circle_r - 1, circle_x + circle_r + 1, 150 )
y = np.linspace(circle_y - circle_r - 1, circle_y + circle_r + 1, 150 )
 
a, b = np.meshgrid( x , y )

C = (a - circle_x) ** 2 + (b - circle_y) ** 2 - circle_r**2
 
figure, axes = plt.subplots()
 
axes.contour( a , b , C , [0] )
axes.set_aspect( 1 )

plt.plot(8.21, 0, 'bo')
plt.plot(0.34, 6.62, 'bo')
plt.plot(5.96, -1.12, 'bo')

 
plt.title( 'Center-Radius form Circle' )
plt.show()